## Loading libraries

In [1]:
import xarray as xr
import pandas as pd
import numpy as np

## Load FAO-LME masks
There are two horizontal resolutions available: $1^{\circ}$ and $0.25^{\circ}$.

In [111]:
gfdl_025 = xr.open_dataset('/g/data/vf71/shared_resources/fao_lme_masks/gfdl-mom6-cobalt2_fao-major_lme_15arcmin_global.nc').id
gfdl_025 = gfdl_025.rename({'longitude': 'lon', 'latitude': 'lat'})
gfdl_025

<xarray.DataArray 'id' (lat: 720, lon: 1440)> Size: 4MB
[1036800 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * lat      (lat) float64 6kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
Attributes:
    long_name:     FAO LME regions
    grid_mapping:  crs

In [107]:
gfdl = xr.open_dataset('/g/data/vf71/shared_resources/fao_lme_masks/gfdl-mom6-cobalt2_fao-major_lme_60arcmin_global.nc').id
gfdl = gfdl.rename({'longitude': 'lon', 'latitude': 'lat'})
gfdl

<xarray.DataArray 'id' (lat: 180, lon: 360)> Size: 259kB
[64800 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 3kB -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 1kB 89.5 88.5 87.5 86.5 ... -86.5 -87.5 -88.5 -89.5
Attributes:
    long_name:     FAO LME regions
    grid_mapping:  crs

## Extracting unique IDs
Regardless of the resolution, we have 83 FAO-LME regions. We will use one of the masks to extract these unique IDs.

In [131]:
id_reg = np.unique(gfdl_025.values)
id_reg = id_reg[np.isfinite(id_reg)]
id_reg

array([ 21.,  27.,  31.,  34.,  41.,  47.,  48.,  51.,  57.,  58.,  61.,
        67.,  71.,  77.,  81.,  87.,  88., 101., 102., 103., 104., 105.,
       106., 107., 108., 109., 110., 111., 112., 113., 114., 115., 116.,
       117., 118., 119., 120., 121., 122., 123., 124., 125., 126., 127.,
       128., 129., 130., 131., 132., 133., 134., 135., 136., 137., 138.,
       139., 140., 141., 142., 143., 144., 145., 146., 147., 148., 149.,
       150., 151., 152., 153., 154., 155., 156., 157., 158., 159., 160.,
       161., 162., 163., 164., 165., 166.], dtype=float32)

## Load area of grid cells
These files match the two horizontal resolutions available for the FAO-LME masks: $1^{\circ}$ and $0.25^{\circ}$.

In [121]:
gfdl_025_area = xr.open_dataarray('/g/data/vf71/shared_resources/grid_cell_area_ESMs/cellarea_15arcmin.nc')
gfdl_025_area

<xarray.DataArray 'cell_area' (lat: 720, lon: 1440)> Size: 8MB
[1036800 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * lat      (lat) float64 6kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
Attributes:
    standard_name:  area
    long_name:      area of grid cell
    units:          m2

In [108]:
gfdl_area = xr.open_dataarray('/g/data/vf71/shared_resources/grid_cell_area_ESMs/cellarea_onedeg.nc')
gfdl_area

<xarray.DataArray 'cell_area' (lat: 180, lon: 360)> Size: 518kB
[64800 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 3kB -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 1kB 89.5 88.5 87.5 86.5 ... -86.5 -87.5 -88.5 -89.5
Attributes:
    standard_name:  area
    long_name:      area of grid cell
    units:          m2

## Calculating weights per FAO-LME region
To do, we will divide the grid cells within each FAO-LME region by its total area.

In [153]:
gfdl_025_weights = xr.zeros_like(gfdl_025_area)
for i in id_reg:
    aoi = gfdl_025.where(gfdl_025 == i)
    gfdl_025_weights = xr.where(gfdl_025 == i, (aoi/aoi.sum()), gfdl_025_weights)

gfdl_025_weights = xr.where(np.isnan(gfdl_025), np.nan, gfdl_025_weights)

In [154]:
gfdl_weights = xr.zeros_like(gfdl_area)
for i in id_reg:
    aoi = gfdl.where(gfdl == i)
    gfdl_weights = xr.where(gfdl == i, (aoi/aoi.sum()), gfdl_weights)

gfdl_weights = xr.where(np.isnan(gfdl), np.nan, gfdl_weights)

### Saving weights per region

In [125]:
gfdl_025_weights.to_netcdf('/g/data/vf71/shared_resources/fao_lme_masks/gfdl-mom6-cobalt2_fao-major_lme_area-weights_15arcmin_global.nc')
gfdl_weights.to_netcdf('/g/data/vf71/shared_resources/fao_lme_masks/gfdl-mom6-cobalt2_fao-major_lme_area-weights_60arcmin_global.nc')

## Transforming gridded data to data frames

In [126]:
weights_df_025 = pd.merge(gfdl_025_weights.to_series().reset_index(), 
                          gfdl_025.to_series().reset_index(),
                          on = ['lat', 'lon'])
weights_df_025.rename({0: 'weight'}, axis = 1, inplace = True)
weights_df_025

,lat,lon,weight,id
0,89.875,-179.875,0.00002,165.0
1,89.875,-179.625,0.00002,165.0
2,89.875,-179.375,0.00002,165.0
3,89.875,-179.125,0.00002,165.0
4,89.875,-178.875,0.00002,165.0
...,...,...,...,...
1036795,-89.875,178.875,NaN,NaN
1036796,-89.875,179.125,NaN,NaN
1036797,-89.875,179.375,NaN,NaN
1036798,-89.875,179.625,NaN,NaN


In [129]:
weights_df = pd.merge(gfdl_weights.to_series().reset_index(), 
                      gfdl.to_series().reset_index(),
                      on = ['lat', 'lon'])
weights_df.rename({0: 'weight'}, axis = 1, inplace = True)
weights_df

,lat,lon,weight,id
0,89.5,-179.5,0.000315,165.0
1,89.5,-178.5,0.000315,165.0
2,89.5,-177.5,0.000315,165.0
3,89.5,-176.5,0.000315,165.0
4,89.5,-175.5,0.000315,165.0
...,...,...,...,...
64795,-89.5,175.5,NaN,NaN
64796,-89.5,176.5,NaN,NaN
64797,-89.5,177.5,NaN,NaN
64798,-89.5,178.5,NaN,NaN


### Saving data frames

In [127]:
weights_df_025.to_csv('/g/data/vf71/shared_resources/fao_lme_masks/gfdl-mom6-cobalt2_fao-major_lme_area-weights_15arcmin_global.csv')

In [130]:
weights_df.to_csv('/g/data/vf71/shared_resources/fao_lme_masks/gfdl-mom6-cobalt2_fao-major_lme_area-weights_60arcmin_global.csv')